modelle/module erklären

In [ ]:
# required: pip install deepface matplotlib
import os

from deepface import DeepFace
import cv2
import matplotlib.pyplot as plt
from fer import FER
from tabulate import tabulate  # Installieren Sie die tabulate-Bibliothek mit "pip install tabulate"

Teil 1
hier können eigene ordner eingefügt werden, die Bilder mit gesichtern enthalten

In [ ]:

# Pfad zum Ordner mit den Bildern
folder_path = 'IMG_1001.JPG'


In diesem Code werden viele Schritte durchgeführt, um Gesichter in Bildern zu erkennen und dann festzustellen, wie ähnlich oder verschieden diese Gesichter sind.

Mit der for-Schleife betrachten wir jedes einzelne Bild im Ordner.
Dann nehmen wir zwei Bilder und erstellen ein "Bildpaar" aus diesen beiden Bildern, um zu überprüfen, wie ähnlich oder verschieden sie sind.
Erkenne Gesichter: Wir schauen uns jedes Bild in diesem Bildpaar an und versuchen, Gesichter darauf zu finden.
Durchführen der Gesichtsverifikation: Wenn wir in beiden Bildern Gesichter finden, führen wir einen speziellen Prozess namens "Gesichtsverifikation" durch. Dieser Prozess untersucht,
wie ähnlich oder verschieden die Gesichter in den Bildern sind. Wir verwenden ein Modell namens "VGG-Face" für diese Überprüfung.
Im Detail passiert folgendes: hier noch erklärung was under the hood passiert, also vorverarbeitung durch vgg face, und repräsentation eines gesichts als vektors durch vgg face modell + darauf hinweisen welche anderen modelle man in deepface auswählen kann

In [ ]:
# Liste, um die Ergebnisse der Gesichtsverifikation zu speichern
results = []

# Durchlaufe alle Bilder im Ordner
files = os.listdir(folder_path)
for i in range(len(files)):
    for j in range(i + 1, len(files)):
        # Pfade zu den Bildern erstellen
        img1_path = os.path.join(folder_path, files[i])
        img2_path = os.path.join(folder_path, files[j])

        # Gesichter in den Bildern erkennen (mit DeepFace)
        detected_faces_img1 = DeepFace.extract_faces(img1_path, enforce_detection=False)
        detected_faces_img2 = DeepFace.extract_faces(img2_path, enforce_detection=False)

        if detected_faces_img1 is not None and detected_faces_img2 is not None:
            # Verifikationsprozess durchführen
            result = DeepFace.verify(img1_path=img1_path, img2_path=img2_path, model_name='VGG-Face', enforce_detection=False)

            # Ergebnis zur Ergebnisliste hinzufügen (als Liste, nicht als Tupel)
            results.append([files[i], files[j], result["distance"], result["verified"]])
        else:
            # Wenn Gesichter nicht erkannt wurden, geben Sie den Bildtitel aus
            print(f"Gesichter in {files[i]} und {files[j]} wurden nicht erkannt.")


In [ ]:
# Erstellen Sie eine Tabelle mit den Ergebnissen
table_headers = ["Bild 1", "Bild 2", "Distanz", "Wahrheitswert"]
table_data = results

# Tabelle anzeigen
print(tabulate(table_data, headers=table_headers, tablefmt="pretty"))



hier können wir den schwellenwert selbst festlegen und ergebnisse ausgeben lassen

In [ ]:
# Festlegen des Schwellenwerts für die Gesichtsverifikation
threshold = 0.6  # Ändern Sie diesen Schwellenwert nach Bedarf

# Überprüfen Sie, ob der Wahrheitswert basierend auf dem Schwellenwert True oder False ist
updated_table_data = []
for row in table_data:
    distance = row[2]
    is_verified = distance < threshold
    updated_table_data.append([row[0], row[1], distance, is_verified])

# Aktualisierte Tabelle anzeigen
print("\nAktualisierte Tabelle mit dem Schwellenwert:")
print(tabulate(updated_table_data, headers=table_headers, tablefmt="pretty"))


Teil 2

In [ ]:
# Emotion-Recognizer-Objekt erstellen
emotion_recognizer = FER()

# Gesichtserkennungskaskade laden
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

# Flag für die Anzeige der Emotionen und Altersschätzung
show_emotions = True
show_age = True

In [ ]:
# Kamera initialisieren
camera = cv2.VideoCapture(0)

while True:
    # Bild vom Kamerastream lesen
    ret, frame = camera.read()

    # Gesichtserkennung im Bild durchführen
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        if show_emotions:
            # Gesichtsausschnitt extrahieren
            face_image = frame[y:y + h, x:x + w]

            # Emotionen im Gesicht erkennen
            result = emotion_recognizer.detect_emotions(face_image)

            if len(result) > 0:
                # Emotionen auf dem Bild anzeigen
                emotions = result[0]['emotions']
                dominant_emotion = max(emotions, key=emotions.get)
                cv2.putText(frame, dominant_emotion, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
                cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

        if show_age:
            # Alter schätzen
            age = int(w / 10)  # Einfache Schätzung basierend auf Gesichtsbreite
            cv2.putText(frame, f"AGE: {age}", (x, y + h + 30), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)

        else:
            # Nur den grünen Kasten anzeigen
            cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)


In [ ]:


    # Livestream anzeigen
    cv2.imshow('Emotion and Age Detection', frame)


In [ ]:

    # Tastenabfrage
    key = cv2.waitKey(1)
    if key == ord('q') or key == ord('Q'):  # 'q' drücken, um die Schleife zu beenden
        break
    elif key == ord('e') or key == ord('E'):  # 'e' drücken, um die Emotionen ein- oder auszublenden
        show_emotions = not show_emotions
    elif key == ord('a') or key == ord('A'):  # 'a' drücken, um das Alter ein- oder auszublenden
        show_age = not show_age

In [ ]:

# Kameraressourcen freigeben
camera.release()
cv2.destroyAllWindows()